**개요 : 영윤의 가장 최근 데이터를 이용해서 재료, 설명, 레시피에 따른 추천 음식을 출력하고 비교해봄**

In [255]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [256]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/220721_정영윤_ver3 (영문, 병합 번역).csv')

In [257]:
df.head()

,id,카테고리명,분류기준,식품명,설명,ingre,recipe
0,1,양식,그라탕류,라따뚜이그라탕,"It is a gratin using ratatouille, a traditiona...","Tomato, eggplant, zucchini, penne, tomato sauc...","Prepare tomatoes, eggplant, and zucchini by cu..."
1,2,간식,토스트류,햄치즈토스트,Sliced ​​ham and cheese are placed on bread an...,"White bread, sliced ​​ham, cheddar cheese, can...",Prepare the cabbage and carrots by slicing the...
2,3,밑반찬,건어물류,마른미역자반,It is a side dish of dried seaweed fried in oi...,"Dried seaweed, perilla oil, cooking oil, sugar...",Heat perilla oil and cooking oil in a pan. Whe...
3,4,한식,찌개류,돼지고기된장찌개,It is a stew made by slicing pork and putting ...,"Pork leg meat, green onion, onion, zucchini, r...",Cut the onion and pumpkin into appropriate siz...
4,5,간식,빵류,고구마치즈빵,A sweet-tasting bread topped with boiled sweet...,"Steamed sweet potato, mayonnaise, honey, mozza...","Mix flour, milk, melted butter and salt in a b..."


In [258]:
df.shape

(2381, 7)

In [259]:
df = df.dropna()    # 결손 데이터
df.shape

(2343, 7)

In [260]:
df.reset_index(inplace=True)

**레시피에 대한 피처 벡터화**

In [261]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')     # 크게 의미없는 단어들을 제외해준다.
tfidf_matrix = tfidf.fit_transform(df['recipe'])
tfidf_matrix.shape

(2343, 3462)

**설명에 대한 피처 벡터화**

In [262]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer(stop_words='english')     # 크게 의미없는 단어들을 제외해준다.
tfidf_matrix2 = tfidf2.fit_transform(df['설명'])
tfidf_matrix2.shape

(2343, 2023)

In [263]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim_recipe = linear_kernel(tfidf_matrix, tfidf_matrix)     # 코사인 유사도 계산
cosine_sim_recipe

array([[1.        , 0.06774809, 0.01612975, ..., 0.04568099, 0.02942222,
        0.1179793 ],
       [0.06774809, 1.        , 0.06186918, ..., 0.03076956, 0.04576709,
        0.15774536],
       [0.01612975, 0.06186918, 1.        , ..., 0.00779477, 0.06149775,
        0.11839264],
       ...,
       [0.04568099, 0.03076956, 0.00779477, ..., 1.        , 0.11404926,
        0.02412688],
       [0.02942222, 0.04576709, 0.06149775, ..., 0.11404926, 1.        ,
        0.04042471],
       [0.1179793 , 0.15774536, 0.11839264, ..., 0.02412688, 0.04042471,
        1.        ]])

In [264]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim_sum = linear_kernel(tfidf_matrix2, tfidf_matrix2)     # 코사인 유사도 계산
cosine_sim_sum

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.07437211,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.180337  ,
        0.        ],
       [0.        , 0.07437211, 0.        , ..., 0.180337  , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [265]:
indices = pd.Series(df.index, index=df['식품명']).drop_duplicates()    # 중복 제거
indices

식품명
라따뚜이그라탕        0
햄치즈토스트         1
마른미역자반         2
돼지고기된장찌개       3
고구마치즈빵         4
            ... 
홍루이젠        2338
불고기우동샐러드    2339
바나나경단       2340
바나나로띠       2341
닭가슴살채소덮밥    2342
Length: 2343, dtype: int64

In [266]:
# 음식명을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 음식 목록 반환
def get_recommendations(title, cosine_sim):

  # 음식명을 통해서 전체 데이터 기준 그 음식의 index 값을 얻기
  idx = indices[title]

  # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 코사인 유사도 기준으로 내림차순 정렬
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # 자기 자신을 제외한 10개의 추천 음식을 슬라이싱
  sim_scores = sim_scores[1:11]

  # 추천 음식목록 10개의 인덱스 정보 추출
  movie_indices = [i[0] for i in sim_scores]

  # 인덱스 정보를 통해 음식명 추출
  return df['식품명'].iloc[movie_indices]

In [267]:
get_recommendations('햄치즈토스트', cosine_sim_recipe)

1750        양배추샌드위치
1503        달걀채소토스트
1970         모닝샌드위치
1745     참치마요네즈샌드위치
2049       베이컨치즈토스트
109     몬테크리스토 샌드위치
1452         프렌치토스트
62          식빵 누네띠네
44            밤잼토스트
143         치즈갈릭브레드
Name: 식품명, dtype: object

In [268]:
get_recommendations('햄치즈토스트', cosine_sim_sum)

2338           홍루이젠
109     몬테크리스토 샌드위치
1312         몬테크리스토
1016        레어치즈케이크
1520          허니브레드
982           식빵이자냐
1735        새우그라탕덮밥
1477          크림통식빵
286           닭날개구이
1219            허니윙
Name: 식품명, dtype: object

In [269]:
get_recommendations('고구마치즈빵', cosine_sim_recipe)

46      제티체크쿠키&제티설기
428            감자피자
173             소금빵
1095            공갈빵
1506       고구마베이컨말이
2030          참깨크래커
2328           시나몬롤
624             모닝빵
1442         감자찹쌀구이
1261           사과파이
Name: 식품명, dtype: object

In [270]:
get_recommendations('고구마치즈빵', cosine_sim_sum)

1791          더블치즈토스트
1867            치즈마늘빵
2049         베이컨치즈토스트
104              프로틴빵
750               구름빵
824     리얼고구마빵&리얼 감자빵
2085            하이토스트
1477            크림통식빵
2022           고구마치즈볼
109       몬테크리스토 샌드위치
Name: 식품명, dtype: object

In [271]:
get_recommendations('닭가슴살채소덮밥', cosine_sim_recipe)

1396       닭가슴살채소볶음
98            오트밀카레
635       후루룩닭고기쌀국수
102           순두부인헬
1576         닭가슴살쌈밥
232           오트밀닭죽
1768      닭가슴살오픈토스트
1604    통바게트양배추샌드위치
1851         감자칩카나페
2061        치킨김치볶음밥
Name: 식품명, dtype: object

In [272]:
get_recommendations('닭가슴살채소덮밥', cosine_sim_sum)

395          두부김밥
1218        된장리조또
563         김치볶음밥
533           두부밥
205      곤약새우버섯솥밥
2098         오야꼬동
497           곤약밥
635     후루룩닭고기쌀국수
156          꼬막덮밥
1896     데리야끼치킨덮밥
Name: 식품명, dtype: object

**재료를 기준으로 음식을 추천해줌**

In [273]:
df['ingre']

0       Tomato, eggplant, zucchini, penne, tomato sauc...
1       White bread, sliced ​​ham, cheddar cheese, can...
2       Dried seaweed, perilla oil, cooking oil, sugar...
3       Pork leg meat, green onion, onion, zucchini, r...
4       Steamed sweet potato, mayonnaise, honey, mozza...
                              ...                        
2338    Bread, egg, sliced ​​ham, sliced ​​cheese, may...
2339    Beef for bulgogi, udon noodles, lettuce, soy s...
2340    Banana, glutinous rice flour (dry), salt, cast...
2341    Banana, tortilla, egg, butter, condensed milk,...
2342    Boiled chicken breast, cabbage, onion, carrot,...
Name: ingre, Length: 2343, dtype: object

In [274]:
type(df['ingre'][0])

str

**재료 개수를 두고 실험을 해봤는데 가장 납득이 가는 재료 숫자가 4개라는 결론이 나옴**

In [275]:
def ingredient(x):
  x = x.split(',')[:4]
  return ' '.join(x)

In [276]:
# df['ingre'] = df['ingre'].apply(lambda x : x.split(','))
# df['ingre'] = df['ingre'].apply(lambda x : x[:5])
# df['ingre'] = df['ingre'].apply(lambda x : ' '.join(x))
df['ingre'] = df['ingre'].apply(ingredient)
df['ingre']

0                       Tomato  eggplant  zucchini  penne
1       White bread  sliced ​​ham  cheddar cheese  can...
2          Dried seaweed  perilla oil  cooking oil  sugar
3             Pork leg meat  green onion  onion  zucchini
4       Steamed sweet potato  mayonnaise  honey  mozza...
                              ...                        
2338            Bread  egg  sliced ​​ham  sliced ​​cheese
2339    Beef for bulgogi  udon noodles  lettuce  soy s...
2340    Banana  glutinous rice flour (dry)  salt  cast...
2341                        Banana  tortilla  egg  butter
2342        Boiled chicken breast  cabbage  onion  carrot
Name: ingre, Length: 2343, dtype: object

In [277]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
count_matrix3 = count.fit_transform(df['ingre'])
count_matrix3

<2343x1023 sparse matrix of type '<class 'numpy.int64'>'
	with 14521 stored elements in Compressed Sparse Row format>

In [278]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_ingre = cosine_similarity(count_matrix3, count_matrix3)
cosine_sim_ingre

array([[1.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 1.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.1767767,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.1767767, 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]])

In [279]:
get_recommendations('데리야끼치킨덮밥', cosine_sim_ingre)

2263     코코넛치킨카레
1029       간장꼬꼬밥
188     치즈닭갈비볶음밥
1151        불닭볶음
1165        닭장떡국
2222        떡닭떡닭
1602       인절미치킨
1655       닭맥적구이
45         오렌지치킨
130          오징어
Name: 식품명, dtype: object

In [280]:
get_recommendations('햄치즈토스트', cosine_sim_ingre)

2049       베이컨치즈토스트
1312         몬테크리스토
272        오믈렛컵&피자컵
477           구름토스트
1874          크로크마담
447          양파햄치즈빵
2338           홍루이젠
1839         치즈폭탄피자
109     몬테크리스토 샌드위치
219           피자토스트
Name: 식품명, dtype: object

In [281]:
get_recommendations('고구마치즈빵', cosine_sim_ingre)

1591      단호박치즈구이
2022       고구마치즈볼
802         고구마경단
831      아코디언꿀고구마
1401      오지치즈후라이
695        고구마핫도그
1878       고구마말랭이
1506     고구마베이컨말이
1532    훈제오리단호박구이
1636     인절미고구마맛탕
Name: 식품명, dtype: object